## Wave equation using finite differences and method of lines. 

### Oscar Reula

In [ ]:
using Plots
using OrdinaryDiffEq  #Esta es solo una parte del paquete DifferentialEquations
#using DifferentialEquations
using Plots
using LinearAlgebra
#import Pkg; Pkg.add("BandedMatrices")
using BandedMatrices
using SparseArrays
using LaTeXStrings

We shall be solving the simplest hyperbolic equation,

$$
u_t = u_x,
$$

representing a wave moving to the left with speed 1.

If we know the value of $u(t,x)$ at $t=t_0$, $u_0(x)$, then we know the value of $u(t,x)$ for all $t$. It is given by:

$$
u(t,x) = u_0(x+(t-t_0))
$$

**Exercise:** Check that the above function satisface the equation. 


In the example that follows, we shall use as initial data the following function:

$$
u_0(x) = \left\{ 
            \begin{array}{l}
            (x-x_l)^p*(x-x_r)^p/(\frac{x_r-x_l}{2})^{2p} & x_l < x < x_r \\
            0 & x<x_l \; \text{or} \; x_r < x
            \end{array}
            \right.
$$ 

And we shall solve the equations in the interval $[0,L]$, with $L=1$. There we shall impose periodic boundary conditions, that is $u(L) = u(0)$.

In [ ]:
function u0(x,(xl, xr, p))
    if x > xl && x < xr
        return (x-xl)^p * (xr-x)^p / (xr-xl)^(2p) * 2^(2p)
    else
        return 0
    end
end
p=8
plot(x->u0(x,(0.4,0.6,p)), xlim=(0,1), label="u0", title="initial data with p=$p")

The plot of this function (solving for the interval $[0,1]$ **with periodic boundary conditions**) is:

In [ ]:
N = 20
dt = 1/N
α = 0.1
plt = plot(title="evolution of L(u_0)", legend=false)
for i in 0:N
    plot!(x-> u0(mod(x + i*dt,1),(0.4,0.6,8)).+ α*i)
end
plt


We shall use **the method of lines** and **finite-differences** approximations. This means that shall first (conceptually) 
discretize only the space at evenly spaced points, $x_i = x_0 + dx*(i-1), \; i=1..N$. For simplicity we shall take de domain to be periodic with period $L$, so that $dx = L/N$. 
Thus we shall consider a vector $v=v(t)$ of length $N$ representing the values of some approximate solution $\tilde{v}(t,x)$ at whose points. That is, 

$$
\tilde{v}(t,x_i) = v_i(t)
$$

We shall also approximate the equation by a finite-difference operator, $D_x$, on the space direction, so as to consider the following system:

\begin{equation}
v_t = D_x\; v.
\end{equation}

This way we get a system of ordinary differential equations of dimension $N$. We then proceed to approximate this system using an appropriate ODE integrator. This way we end up with a discretization in space and time.


In [ ]:
N = 8 # space points
M = 5 # time points
L = 2
T = 1

function plot_grid(X,Y,Nx::Int,Ny::Int,xaxis::String,yaxis::String)
dx = X/N
dy = Y/(M-1)
x = [dx*(i-1) for i in 1:N]
#s = [[(dx*(i-1),dt*(j-1)) for i in 1:N] for j in 1:M]
y = [[(dy*(j-1)) for i in 1:N] for j in 1:M]
plt = plot(legend=false, xlabel=xaxis, ylabel=yaxis, xlim=(-0.1,2), aspectratio=1, ylim=(-0.02,1.1))
for j in 1:M
    scatter!(x,y[j])
    plot!([x; 2],[y[j]; dy*(j-1)], lc=:red)
end
for i in 1:N
    plot!([x[i];x[i]],[0.0;Y+0.1], lc=:blue)
end
return plt
end

plot_grid(L,T,N,M,"x","t")

Grid for $L=2$, $N=8$, $T=1$, $M=5$

Now we add some parameters for the simulation. Some are set to arbitrary values, just to indicate that you can add them if needed. $N$ is the number of point of our space discretization. We are going to be solving a *periodic* problem, so we start with point 1 and finish with point $N$, the point $N+1$ is identified with the point $1$ and so on. 

In [ ]:
#N = 8 # number of space-points for the space discretization
N = 200
L = 1. # the space interval #
dx = L/N
T = 1. # final time integratios
dT = 1. *dx # we take a dt around the size of dx/speed_max, 
            # so that the algorith is stable, the CFL condition.
a = 1.
α = 1.
p = (a,1.0,dx) # some parameters a,dx

v0 = zeros(N,1) #the field discretizations, $u(t,(i-1)*dx)$ and $v(t,i))$
x = zeros(N); # the x coordinate at those points, needed to prescribe initial data.

The restriction to the grid points of the initial data becomes:

In [ ]:
x = [dx*(i-1) for i in 1:N]
u00(x) = u0(x,(0.4,0.6,8))
v0[:,1] = u00.(x)
plot(x,u00.(x), xlim=(0,1), label="u0", title="initial data with p=$p")
scatter!(x,v0, label="v0", ms=2)

We now define the finite-difference schemes. They are implemented as matrices multiplyting the vector solution components. The first two cases are for the second order scheme. They differ in the use of sparce matrices. This is important for efficient, and large systems, here, for the small systems (1D) we are implementing, it does not seems to be of importance. 

Here we use a veri simple finite difference operator:

**D_2_per** is centered FD of second order which is adapted to the periodic case, 

$$
Dv_i = \frac{v_{i+1} - v_{i-1}}{2\Delta x}
$$ 
as we shall see this is not very accurate, only second order (in $\Delta x)$, that is 

$$
\frac{du}{dx} - Du = \mathcal{O}(\Delta x^2)
$$

Its expression is: (for simplicity we do not divide by $\Delta x$)

In [ ]:
D_2_per = Array(Tridiagonal([-0.5 for i in 1:N-1],[0.0 for i in 1:N],[0.5 for i in 1:N-1]))
D_2_per[1,end] = -0.5
D_2_per[end,1] = 0.5

In [ ]:
D_2_per

We define now the rhs of the equations in the method of lines, that is, the space discretization. These versions are for efficiency, and for further modifications. 

In [ ]:
function F2!(dr,r,p,t)
    # second order version
    a,α,dx = p
    h = 1. /dx
    u = @view r[:,1]
    du = @view dr[:,1]
    Du = h * D_2_per * u
    @. du = Du
end


We define now ODE problem,  

In [ ]:
prob2 = ODEProblem(F2!,v0,(0.0,T),p);

We now solve them:

In [ ]:
sol2 = solve(prob2,RK4(),dt=dT,adaptive=false);

Finally we plot the solutions at different times

In [ ]:
plot([v0[:,1],sol2(T*0.0)[:,1],sol2(T*0.2)[:,1],sol2(T*0.3)[:,1],sol2(T*1)[:,1]])

In [ ]:
anim = @animate for i ∈ 1:20
    plot(x,sol2(T*0.05*i)[:,1])
end

gif(anim, "wave_anim_fps10.gif", fps = 10)
    

In [ ]:
plot(x,v0[:,1])
plot!(x,sol2(T)[:,1])
#plot(x,sol.u)

### Alternatives (run with N=2000)

#### Other time-Integrators

What happens if we try with Euler?

In [ ]:
sol2_E = solve(prob2,Euler(),dt=dT,adaptive=false);

In [ ]:
sol2_E = solve(prob2,Euler(),dt=dT/10,adaptive=false);

In [ ]:
plot([v0[:,1],sol2_E(T*0.0)[:,1],sol2_E(T*0.2)[:,1],sol2_E(T*0.3)[:,1],sol2_E(T*1)[:,1]])

In [ ]:
anim = @animate for i ∈ 1:20
    plot(x,sol2_E(T*0.02*i)[:,1])
end

gif(anim, "wave_anim_E_fps5.gif", fps = 5)

In [ ]:
plot([v0[:,1],sol2_E(T*0.3)[:,1],sol2_E(T*0.32)[:,1],sol2_E(T*0.34)[:,1]])

Since Euler is not stable the simulation quickly becomes inaccurate.

#### Big time step

Another way to get instabilities is to get out of the stability region by taking a too big time step. We go back to RK4 and try with a bigger time step.

In [ ]:
sol2_B = solve(prob2,RK4(),dt=dT*2.9,adaptive=false);
plot([v0[:,1],sol2_B(T*0.3)[:,1],sol2_B(T*0.7)[:,1],sol2_B(T*1.0)[:,1]])

#### Other finite differences

We shall use other finite difference operators, one taking a forward difference, namely, 

$$
D_+(v)_i = \frac{v_{i+1} - v_i}{\Delta x}
$$

and another taking a backward difference:

$$
D_-(v)_i = \frac{v_{i} - v_{i-1}}{\Delta x}
$$

In [ ]:
D_p_per = Array(Bidiagonal([-1.0 for i in 1:N],[1 for i in 1:N-1],:U))
D_p_per[end,1] = 1.0


D_m_per = Array(Bidiagonal([1.0 for i in 1:N],[-1 for i in 1:N-1],:L))
D_m_per[1,end] = -1.0

#D_p_per
#D_m_per

In [ ]:
function Fp!(dr,r,p,t)
    # forward first order version
    a,α,dx = p
    h = 1. /dx
    u = @view r[:,1]
    du = @view dr[:,1]
    Du = h * D_p_per * u
    @. du = Du
end

function Fm!(dr,r,p,t)
    # forward first order version
    a,α,dx = p
    h = 1. /dx
    u = @view r[:,1]
    du = @view dr[:,1]
    Du = h * D_m_per * u
    @. du = Du
end

In [ ]:
prob_p = ODEProblem(Fp!,v0,(0.0,T),p);
prob_m = ODEProblem(Fm!,v0,(0.0,T),p);

In [ ]:
sol_p = solve(prob_p,RK4(),dt=dT,adaptive=false);
plot([v0[:,1],sol_p(T*0.3)[:,1],sol_p(T*0.7)[:,1],sol_p(T*1.0)[:,1]])

In [ ]:
sol_m = solve(prob_m,RK4(),dt=dT,adaptive=false);
plot([v0[:,1],sol_m(T*0.3)[:,1],sol_m(T*0.7)[:,1],sol_m(T*1.0)[:,1]])